In [259]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import csv
import re

**Reading in the Data**

In [ ]:
# part 1
r = requests.get("http://www.ufcstats.com/statistics/events/completed?page=all")
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_ = 'b-statistics__section' )
pages = s.find_all('a', class_ = 'b-link b-link_style_black')
hrefs_start = [a['href'] for a in pages]

stats_url = []
fighter1 = []
fighter2 = []
weight = []
method = []
rounds = []
times = []
dates = []
locations = []
event = []
failed_urls = []
count = 0

for url in hrefs_start:
    try:
        time.sleep(1)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')
        rows = rows[1:]
        for row in rows:
            cols = row.find_all('td')
            data = [col.get_text(strip=True) for col in cols][6:]
            weight.append(data[0])
            method.append(data[1])
            rounds.append(data[2])
            times.append(data[3])
            fighters = row.find_all('a', class_ ="b-link b-link_style_black")
            names = [a.get_text(strip = True) for a in fighters]
            fighter1.append(names[0])
            fighter2.append(names[1])
            stat_link = row.get('data-link')
            
            if stat_link != '' or stat_link is not None:
                stats_url.append(row.get('data-link'))
            else: stats_url.append("No Link")

            location = soup.find_all('li', class_ = 'b-list__box-list-item')
            locations.append(location[1].get_text(strip=True)[9:])
            date = soup.find('li', class_ = 'b-list__box-list-item')
            dates.append(date.get_text(strip=True)[5:])
            event.append(soup.find('h2').get_text(strip=True))
        count += 1
        print(f"Success Count: {count}")
    except requests.exceptions.HTTPError as e:
        print("HTTP Error: ", e)
        failed_urls.append(url)
    except requests.exceptions.RequestException as e:
        print("Request Failed: ", e)
        failed_urls.append(url)



#with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(get_event_details, hrefs_start)

Success Count: 1
Success Count: 2
Success Count: 3
Success Count: 4
Success Count: 5
Success Count: 6
Success Count: 7
Success Count: 8
Success Count: 9
Success Count: 10
Success Count: 11
Success Count: 12
Success Count: 13
Success Count: 14
Success Count: 15
Success Count: 16
Success Count: 17
Success Count: 18
Success Count: 19
Success Count: 20
Success Count: 21
Success Count: 22
Success Count: 23
Success Count: 24
Success Count: 25
Success Count: 26
Success Count: 27
Success Count: 28
Success Count: 29
Success Count: 30
Success Count: 31
Success Count: 32
Success Count: 33
Success Count: 34
Success Count: 35
Success Count: 36
Success Count: 37
Success Count: 38
Success Count: 39
Success Count: 40
Success Count: 41
Success Count: 42
Success Count: 43
Success Count: 44
Success Count: 45
Success Count: 46
Success Count: 47
Success Count: 48
Success Count: 49
Success Count: 50
Success Count: 51
Success Count: 52
Success Count: 53
Success Count: 54
Success Count: 55
Success Count: 56
S

In [10]:
len(fighter1)

8276

In [11]:
print(len(fighter1))
print(len(stats_url))
print(len(event))

8276
8276
8276


In [12]:
df = pd.DataFrame({
    "event": event,
    'fighter1': fighter1,
    'fighter2': fighter2,
    'weight': weight,
    'rounds': rounds,
    'times': times,
    'method': method,
    'locations': locations,
    'dates': dates,
    'stats_url': stats_url
})

In [14]:
df.head

<bound method NDFrame.head of                                          event           fighter1  \
0     UFC Fight Night: Whittaker vs. De Ridder  Reinier de Ridder   
1     UFC Fight Night: Whittaker vs. De Ridder           Petr Yan   
2     UFC Fight Night: Whittaker vs. De Ridder    Shara Magomedov   
3     UFC Fight Night: Whittaker vs. De Ridder      Asu Almabayev   
4     UFC Fight Night: Whittaker vs. De Ridder      Bogdan Guskov   
...                                        ...                ...   
8271                         UFC 2: No Way Out       Orlando Wiet   
8272                         UFC 2: No Way Out      Frank Hamaker   
8273                         UFC 2: No Way Out      Johnny Rhodes   
8274                         UFC 2: No Way Out      Patrick Smith   
8275                         UFC 2: No Way Out       Scott Morris   

                  fighter2             weight rounds  times  \
0         Robert Whittaker       Middleweight      5   5:00   
1            Ma

In [16]:
df.shape

(8276, 10)

In [ ]:
#final_df.to_csv('Final_UFC_Events.csv')
#filter out methods that are odd (DQ's, NC's, basically anything other than a decisive victory for one competitor)


In [85]:
# Getting more in-depth stats - part 1
fight_details_links = df['stats_url']

f1 = 0
f2 = 0
f1_kd = 0
f1_sigstr = 0
f1_sigstr_pct = 0
f1_totstr = 0
f1_td = 0
f1_td_pct = 0
f1_subatt = 0
f1_rev = 0
f1_ctrl = 0
row2 = 0
f2_kd = 0
f2_sigstr = 0
f2_sigstr_pct = 0
f2_totstr = 0
f2_td = 0
f2_td_pct = 0
f2_subatt = 0
f2_rev = 0
f2_ctrl = 0
event = 0
stats_url = 0
tot_round = 0

all_data = []

failed_requests = []

connect_timeout = 6
read_timeout = 60
success_count = 0
failure_count = 0


for url in fight_details_links:
    try:
        time.sleep(1.5)
        r = requests.get(url, timeout=(connect_timeout, read_timeout))
        soup = BeautifulSoup(r.content, 'html.parser')
        max_round = soup.find_all("i", class_ = 'b-fight-details__text-item')
        max_round = max_round[2].get_text().split('Rnd')[0].strip()[-1]
        fight_details = soup.find_all('section', class_ = 'b-fight-details__section js-fight-section')
        fight_details = fight_details[1]
        tds = fight_details.find_all('td')
        row1 = []
        event = soup.find('h2').get_text(strip=True)
        for td in tds:
            p_tags = td.find_all('p')
            row1.append(p_tags[0].get_text(strip=True))
        f1 = row1[0]
        f1_kd = row1[1]
        f1_sigstr = row1[2]
        f1_sigstr_pct = row1[3]
        f1_totstr = row1[4]
        f1_td = row1[5]
        f1_td_pct = row1[6]
        f1_subatt = row1[7]
        f1_rev = row1[8]
        f1_ctrl = row1[9]
        row2 = []
        for td in tds:
            p_tags = td.find_all('p')
            row2.append(p_tags[1].get_text(strip=True))
        f2 = row2[0]
        f2_kd = row2[1]
        f2_sigstr = row2[2]
        f2_sigstr_pct = row2[3]
        f2_totstr = row2[4]
        f2_td = row2[5]
        f2_td_pct = row2[6]
        f2_subatt = row2[7]
        f2_rev = row2[8]
        f2_ctrl = row2[9]
        stats_url = url

        all_data.append({
            'fighter1' : f1,
            'fighter2': f2,
            'f1_kd' : f1_kd,
            'f1_sigstr' : f1_sigstr,
            'f1_sigstr_pct' : f1_sigstr_pct,
            'f1_totstr' : f1_totstr,
            'f1_td' : f1_td,
            'f1_td_pct' : f1_td_pct,
            'f1_subatt' : f1_subatt,
            'f1_rev' : f1_rev,
            'f1_ctrl' : f1_ctrl,
            'f2_kd' : f2_kd,
            'f2_sigstr' : f2_sigstr,
            'f2_sigstr_pct' : f2_sigstr_pct,
            'f2_totstr' : f2_totstr,
            'f2_td' : f2_td,
            'f2_td_pct' : f2_td_pct,
            'f2_subatt' : f2_subatt,
            'f2_rev' : f2_rev,
            'f2_ctrl' : f2_ctrl,
            'event' : event,
            'tot_round' : max_round,
            'stats_url' : stats_url
            })
        success_count += 1
        print(f"Succcess count: {success_count}")

    except Exception as e:
        failure_count += 1
        print(f"Failure: {failure_count}")
        failed_requests.append(url)


# with concurrent.futures.ThreadPoolExecutor() as executor:
#     executor.map(get_fight_stats, fight_details_links)

Succcess count: 1
Succcess count: 2
Succcess count: 3
Succcess count: 4
Succcess count: 5
Succcess count: 6
Succcess count: 7
Succcess count: 8
Succcess count: 9
Succcess count: 10
Succcess count: 11
Succcess count: 12
Succcess count: 13
Succcess count: 14
Succcess count: 15
Succcess count: 16
Succcess count: 17
Succcess count: 18
Succcess count: 19
Succcess count: 20
Succcess count: 21
Succcess count: 22
Succcess count: 23
Succcess count: 24
Succcess count: 25
Succcess count: 26
Succcess count: 27
Succcess count: 28
Succcess count: 29
Succcess count: 30
Succcess count: 31
Succcess count: 32
Succcess count: 33
Succcess count: 34
Succcess count: 35
Succcess count: 36
Succcess count: 37
Succcess count: 38
Succcess count: 39
Succcess count: 40
Succcess count: 41
Succcess count: 42
Succcess count: 43
Succcess count: 44
Succcess count: 45
Succcess count: 46
Succcess count: 47
Succcess count: 48
Succcess count: 49
Succcess count: 50
Succcess count: 51
Succcess count: 52
Succcess count: 53
Su

In [91]:
if failed_requests is not None or len(failed_requests) > 0:
    print("Retrying failed requests")
    success_count = 0
    failed_requests_loop = failed_requests
    for url in failed_requests_loop:
        try:
            time.sleep(1)
            r = requests.get(url, timeout=(connect_timeout, read_timeout))
            soup = BeautifulSoup(r.content, 'html.parser')
            max_round = soup.find_all("i", class_ = 'b-fight-details__text-item')
            max_round = max_round[2].get_text().split('Rnd')[0].strip()[-1]
            fight_details = soup.find_all('section', class_ = 'b-fight-details__section js-fight-section')
            fight_details = fight_details[1]
            tds = fight_details.find_all('td')
            row1 = []
            event = soup.find('h2').get_text(strip=True)
            for td in tds:
                p_tags = td.find_all('p')
                row1.append(p_tags[0].get_text(strip=True))
            f1 = row1[0]
            f1_kd = row1[1]
            f1_sigstr = row1[2]
            f1_sigstr_pct = row1[3]
            f1_totstr = row1[4]
            f1_td = row1[5]
            f1_td_pct = row1[6]
            f1_subatt = row1[7]
            f1_rev = row1[8]
            f1_ctrl = row1[9]
            row2 = []
            for td in tds:
                p_tags = td.find_all('p')
                row2.append(p_tags[1].get_text(strip=True))
            f2 = row2[0]
            f2_kd = row2[1]
            f2_sigstr = row2[2]
            f2_sigstr_pct = row2[3]
            f2_totstr = row2[4]
            f2_td = row2[5]
            f2_td_pct = row2[6]
            f2_subatt = row2[7]
            f2_rev = row2[8]
            f2_ctrl = row2[9]
            stats_url = url

            all_data.append({
                'fighter1' : f1,
                'fighter2': f2,
                'f1_kd' : f1_kd,
                'f1_sigstr' : f1_sigstr,
                'f1_sigstr_pct' : f1_sigstr_pct,
                'f1_totstr' : f1_totstr,
                'f1_td' : f1_td,
                'f1_td_pct' : f1_td_pct,
                'f1_subatt' : f1_subatt,
                'f1_rev' : f1_rev,
                'f1_ctrl' : f1_ctrl,
                'f2_kd' : f2_kd,
                'f2_sigstr' : f2_sigstr,
                'f2_sigstr_pct' : f2_sigstr_pct,
                'f2_totstr' : f2_totstr,
                'f2_td' : f2_td,
                'f2_td_pct' : f2_td_pct,
                'f2_subatt' : f2_subatt,
                'f2_rev' : f2_rev,
                'f2_ctrl' : f2_ctrl,
                'event' : event,
                'tot_round' : max_round,
                'stats_url' : stats_url
                })
            success_count += 1
            print(f"Succcess count: {success_count}")
            failed_requests.remove(url)

        except Exception as e:
            failure_count += 1
            print(f"Failure count: {failure_count} - {e}")

# After checking the website for failed urls, it appears that stats don't exist here in these failures.

Retrying failed requests
Failure count: 23 - list index out of range
Failure count: 24 - list index out of range
Failure count: 25 - list index out of range
Failure count: 26 - list index out of range
Failure count: 27 - list index out of range
Failure count: 28 - list index out of range
Failure count: 29 - list index out of range
Failure count: 30 - list index out of range
Failure count: 31 - list index out of range
Failure count: 32 - list index out of range
Failure count: 33 - list index out of range
Failure count: 34 - list index out of range
Failure count: 35 - list index out of range
Failure count: 36 - list index out of range
Failure count: 37 - list index out of range
Failure count: 38 - list index out of range
Failure count: 39 - list index out of range
Failure count: 40 - list index out of range
Failure count: 41 - list index out of range
Failure count: 42 - list index out of range
Failure count: 43 - list index out of range


In [ ]:
# max_round = soup.find_all("i", class_ = 'b-fight-details__text-item')
# max_round = max_round[2].get_text().split('Rnd')[0].strip()[-1]
# max_round

'5'

In [83]:
print(len(f1))
print(len(f2_sigstr))
print(len(event))
print(len(f1_rev))
print(len(stats_url))
print(len(tot_round))

8256
8256
8256
8256
8256
8303


In [93]:
stats_df = pd.DataFrame(all_data)
stats_df.to_csv("Complete Stats.csv")
stats_df.head()

,fighter1,fighter2,f1_kd,f1_sigstr,f1_sigstr_pct,f1_totstr,f1_td,f1_td_pct,f1_subatt,f1_rev,...,f2_sigstr_pct,f2_totstr,f2_td,f2_td_pct,f2_subatt,f2_rev,f2_ctrl,event,tot_round,stats_url
0,Robert Whittaker,Reinier de Ridder,1,66 of 142,46%,70 of 146,0 of 0,---,0,0,...,46%,192 of 282,2 of 15,13%,0,0,9:06,UFC Fight Night: Whittaker vs. De Ridder,5,http://www.ufcstats.com/fight-details/6b8be0ee...
1,Petr Yan,Marcus McGhee,0,81 of 146,55%,130 of 202,1 of 3,33%,0,0,...,37%,66 of 173,0 of 4,0%,0,0,0:30,UFC Fight Night: Whittaker vs. De Ridder,3,http://www.ufcstats.com/fight-details/d4d4ee04...
2,Shara Magomedov,Marc-Andre Barriault,0,93 of 142,65%,103 of 153,0 of 1,0%,0,0,...,50%,60 of 100,2 of 6,33%,0,0,4:30,UFC Fight Night: Whittaker vs. De Ridder,3,http://www.ufcstats.com/fight-details/582d66ce...
3,Asu Almabayev,Jose Ochoa,0,22 of 42,52%,55 of 90,7 of 16,43%,2,0,...,57%,118 of 151,1 of 1,100%,2,0,0:56,UFC Fight Night: Whittaker vs. De Ridder,3,http://www.ufcstats.com/fight-details/887c61da...
4,Nikita Krylov,Bogdan Guskov,0,16 of 34,47%,16 of 34,0 of 0,---,0,0,...,53%,28 of 48,0 of 0,---,0,0,0:17,UFC Fight Night: Whittaker vs. De Ridder,3,http://www.ufcstats.com/fight-details/77f960d8...


In [ ]:
# Filtering out unusual fight results
# unusal_meths = ['CNC', 'M-DEC', 'DQ',  'OverturnedKick', 'DQRear Naked Choke', 'OverturnedTriangle Choke','OverturnedElbows', 'Other']
# final_df2 = final_df[~final_df['method'].isin(unusal_meths)]

In [220]:
complete_df = stats_df.merge(df[['stats_url', 'fighter1', 'fighter2', 'event']], on = ['stats_url', 'fighter1', 'fighter2'], how= "left")

In [221]:
complete_df.columns

Index(['fighter1', 'fighter2', 'f1_kd', 'f1_sigstr', 'f1_sigstr_pct',
       'f1_totstr', 'f1_td', 'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl',
       'f2_kd', 'f2_sigstr', 'f2_sigstr_pct', 'f2_totstr', 'f2_td',
       'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'event_x', 'tot_round',
       'stats_url', 'event_y'],
      dtype='object')

In [222]:
mismatched_winner_df = complete_df[complete_df['event_y'].isnull()]

In [ ]:
complete_df = complete_df[~complete_df['event_y'].isnull()]

In [224]:
# Rows where fighter1 and fighter2 are switched. Switching columns names, concatenating rows to original df and rejoining data to produce final df.
# This is because we are determining winner based on their html location in df - stats_df may have winner rearranged.
len(mismatched_winner_df)

2873

In [225]:
mismatched_winner_df.columns

Index(['fighter1', 'fighter2', 'f1_kd', 'f1_sigstr', 'f1_sigstr_pct',
       'f1_totstr', 'f1_td', 'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl',
       'f2_kd', 'f2_sigstr', 'f2_sigstr_pct', 'f2_totstr', 'f2_td',
       'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'event_x', 'tot_round',
       'stats_url', 'event_y'],
      dtype='object')

In [226]:
mismatched_winner_df = mismatched_winner_df.rename(columns={
'fighter1' : 'fighter2',
'fighter2': 'fighter1',
'f1_kd' : 'f2_kd',
'f1_sigstr' : 'f2_sigstr',
'f1_sigstr_pct' : 'f2_sigstr_pct',
'f1_totstr' : 'f2_totstr',
'f1_td' : 'f2_td',
'f1_td_pct' : 'f2_td_pct',
'f1_subatt' : 'f2_subatt',
'f1_rev' : 'f2_rev',
'f1_ctrl' : 'f2_ctrl',
'f2_kd' : 'f1_kd',
'f2_sigstr' : 'f1_sigstr',
'f2_sigstr_pct' : 'f1_sigstr_pct',
'f2_totstr' : 'f1_totstr',
'f2_td' : 'f1_td',
'f2_td_pct' : 'f1_td_pct',
'f2_subatt' : 'f1_subatt',
'f2_rev' : 'f1_rev',
'f2_ctrl' : 'f1_ctrl',
'event_x' : 'event'
})


In [227]:
mismatched_winner_df = mismatched_winner_df.drop(['event_y'], axis=1)
fixed_mismatch_df = mismatched_winner_df.merge(df, how = 'inner', on = ['fighter1', 'fighter2', 'stats_url'])

In [228]:
fixed_mismatch_df.columns

Index(['fighter2', 'fighter1', 'f2_kd', 'f2_sigstr', 'f2_sigstr_pct',
       'f2_totstr', 'f2_td', 'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl',
       'f1_kd', 'f1_sigstr', 'f1_sigstr_pct', 'f1_totstr', 'f1_td',
       'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl', 'event_x', 'tot_round',
       'stats_url', 'event_y', 'weight', 'rounds', 'times', 'method',
       'locations', 'dates'],
      dtype='object')

In [229]:
complete_df.columns

Index(['fighter1', 'fighter2', 'f1_kd', 'f1_sigstr', 'f1_sigstr_pct',
       'f1_totstr', 'f1_td', 'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl',
       'f2_kd', 'f2_sigstr', 'f2_sigstr_pct', 'f2_totstr', 'f2_td',
       'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'event_x', 'tot_round',
       'stats_url', 'event_y'],
      dtype='object')

In [230]:
complete_df = complete_df.rename(columns={
'event_x' : 'event'
})
complete_df = complete_df.drop(['event_y'], axis=1)
matched_winner = complete_df.merge(df, how='left', on = ['fighter1', 'fighter2', 'stats_url'])

In [231]:
complete_df = pd.concat([fixed_mismatch_df, matched_winner], ignore_index=True)

#complete_df = final_df2.merge(final_stats_join, on = ['stats_url', 'fighter1', 'fighter2'], how="inner")
len(complete_df)

8256

In [240]:
complete_df = complete_df.rename(columns = {'event_x' : 'event'})
complete_df = complete_df.drop(['event_y'], axis=1)

In [241]:
complete_df[['f1_sigstr_landed', 'f1_sigstr_attempt']] = complete_df['f1_sigstr'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_sigstr_landed', 'f2_sigstr_attempt']] = complete_df['f2_sigstr'].str.split(' of ', expand=True).astype(int)

complete_df[['f1_totstr_landed', 'f1_totstr_attempt']] = complete_df['f1_totstr'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_totstr_landed', 'f2_totstr_attempt']] = complete_df['f2_totstr'].str.split(' of ', expand=True).astype(int)

complete_df[['f1_td_landed', 'f1_td_attempt']] = complete_df['f1_td'].str.split(' of ', expand=True).astype(int)
complete_df[['f2_td_landed', 'f2_td_attempt']] = complete_df['f2_td'].str.split(' of ', expand=True).astype(int)

In [242]:
complete_df = complete_df.drop(columns=[
'f1_sigstr', 'f2_sigstr', 'f1_totstr', 'f2_totstr', 'f1_td', 'f2_td'
], axis=1)

In [243]:
complete_df['f1_sigstr_pct'] = complete_df.apply(lambda row: row['f1_sigstr_landed']/row['f1_sigstr_attempt'] * 100 if row['f1_sigstr_attempt'] != 0 else 0, axis=1)
complete_df['f2_sigstr_pct'] = complete_df.apply(lambda row: row['f2_sigstr_landed']/row['f2_sigstr_attempt'] * 100 if row['f2_sigstr_attempt'] != 0 else 0, axis=1)

complete_df['f2_totstr_pct'] = complete_df.apply(lambda row: row['f2_totstr_landed']/row['f2_totstr_attempt'] * 100 if row['f2_totstr_attempt'] != 0 else 0, axis=1)
complete_df['f1_totstr_pct'] = complete_df.apply(lambda row: row['f1_totstr_landed']/row['f1_totstr_attempt'] * 100 if row['f1_totstr_attempt'] != 0 else 0, axis=1)

complete_df['f1_td_pct'] = complete_df.apply(lambda row: row['f1_td_landed']/row['f1_td_attempt'] * 100 if row['f1_td_attempt'] != 0 else 0, axis=1)
complete_df['f2_td_pct'] = complete_df.apply(lambda row: row['f2_td_landed']/row['f2_td_attempt'] * 100 if row['f2_td_attempt'] != 0 else 0, axis=1)

In [245]:
complete_df['rounds'] = complete_df['rounds'].astype(int)

In [248]:
# Get MM:SS time format into just seconds to help aggregate / total time in the fight
def standard_time_format(mmss):
    if mmss == '--':
        return 0
    else: minutes, seconds = map(int, mmss.split(":"))
    return (minutes*60) + seconds

def total_fight_time(row):
    remainder = standard_time_format(row['times'])
    x = row['rounds'] - 1
    return (x*5*60) + remainder

In [249]:
complete_df['f1_ctrl_sec'] = complete_df['f1_ctrl'].apply(standard_time_format)
complete_df['f2_ctrl_sec'] = complete_df['f2_ctrl'].apply(standard_time_format)

In [250]:
complete_df['tot_fight_secs'] = complete_df.apply(total_fight_time, axis=1)

In [251]:
complete_df['more_totstr_landed'] = complete_df.apply(lambda row: 'fighter1' if row['f1_totstr_landed'] > row['f2_totstr_landed'] else ('fighter2' if row['f1_totstr_landed'] < row['f2_totstr_landed'] else "equal"), axis=1)
complete_df['more_totstr_attempt'] = complete_df.apply(lambda row: 'fighter1' if row['f1_totstr_attempt'] > row['f2_totstr_attempt'] else ('fighter2' if row['f1_totstr_attempt'] < row['f2_totstr_attempt'] else "equal"), axis=1)

complete_df['more_sigstr_attempt'] = complete_df.apply(lambda row: 'fighter1' if row['f1_sigstr_attempt'] > row['f2_sigstr_attempt'] else ('fighter2' if row['f1_sigstr_attempt'] < row['f2_sigstr_attempt'] else "equal"),axis=1)
complete_df['more_sigstr_landed'] = complete_df.apply(lambda row: 'fighter1' if row['f1_sigstr_landed'] > row['f2_sigstr_landed'] else ('fighter2' if row['f1_sigstr_landed'] < row['f2_sigstr_landed'] else "equal"),axis=1)


In [252]:
complete_df.to_csv("Complete Stats.csv")

In [263]:
stats_df = pd.read_csv("Complete Stats.csv")

# separate into two tables then append each other to normalize the data
# stats_df.columns
f1_df = stats_df[['event', 'fighter1', 'weight', 'rounds',
       'times', 'method', 'locations', 'dates', 'stats_url', 'f1_kd',
       'f1_sigstr_pct', 'f1_td_pct', 'f1_subatt', 'f1_rev', 'f1_ctrl',
       'f1_sigstr_landed', 'f1_sigstr_attempt', 'f1_totstr_landed', 'f1_totstr_attempt', 'f1_td_landed',
       'f1_td_attempt',
       'f1_totstr_pct', 'f1_ctrl_sec', 'more_totstr_landed',
       'more_totstr_attempt', 'more_sigstr_attempt', 'more_sigstr_landed', 'tot_round', 'tot_fight_secs']].copy()

f1_df["is_winner"] = True
f1_df["fighter_num"] = "fighter1"
f1_df.rename(columns={"fighter1" : "fighter" , 'f1_kd' : 'kd',
       'f1_sigstr_pct' : 'sigstr_pct', 'f1_td_pct' : 'td_pct', 'f1_subatt' : 'subatt', 'f1_rev' : 'rev', 'f1_ctrl' : 'ctrl', 'f1_sigstr_landed' : 'sigstr_landed',
       'f1_sigstr_attempt' : 'sigstr_attempt',
       'f1_totstr_landed' : 'totstr_landed', 'f1_totstr_attempt' : 'totstr_attempt', 'f1_td_landed' : 'td_landed', 'f1_td_attempt' : 'td_attempt', 'f1_totstr_pct' : 'totstr_pct', 'f1_ctrl_sec' : 'ctrl_sec'}, inplace=True)

f2_df = stats_df[['event', 'fighter2', 'weight', 'rounds',
       'times', 'method', 'locations', 'dates', 'stats_url', 'f2_kd',
       'f2_sigstr_pct', 'f2_td_pct', 'f2_subatt', 'f2_rev', 'f2_ctrl', 'f2_sigstr_landed',
       'f2_sigstr_attempt',
       'f2_totstr_landed', 'f2_totstr_attempt', 'f2_td_landed', 'f2_td_attempt', 'f2_totstr_pct', 'f2_ctrl_sec', 'more_totstr_landed',
       'more_totstr_attempt', 'more_sigstr_attempt', 'more_sigstr_landed', 'tot_round', 'tot_fight_secs']].copy()

f2_df["is_winner"] = False
f2_df["fighter_num"] = "fighter2"
f2_df.rename(columns={"fighter2" : "fighter", "f2_kd": "kd",
       'f2_sigstr_pct': 'sigstr_pct', 'f2_td_pct': 'td_pct', 'f2_subatt' : 'subatt', 'f2_rev': 'rev', 'f2_ctrl': 'ctrl', 'f2_sigstr_landed': 'sigstr_landed',
       'f2_sigstr_attempt' : 'sigstr_attempt',
       'f2_totstr_landed' : 'totstr_landed', 'f2_totstr_attempt' : 'totstr_attempt', 'f2_td_landed' : 'td_landed', 'f2_td_attempt' : 'td_attempt', 'f2_totstr_pct' : 'totstr_pct', 'f2_ctrl_sec' : 'ctrl_sec'}, inplace=True)

In [264]:
norm_df = pd.concat([f1_df, f2_df], ignore_index = True)

In [270]:
### Consolidate methods
def consolidate_methods(x):
    if "SUB" in x:
        return "SUB"
    elif "KO" in x:
        return "KO/TKO"
    elif "DEC" in x:
        return "DEC"
    else: 
        return "DQ/CNC/Overturned/Other"

In [271]:
norm_df['method'] = norm_df['method'].apply(consolidate_methods)

In [273]:
norm_df.to_csv("Normalized Stats Table.csv")